# Basic SQLAlchemy SQL Expressions

In [ ]:
import urllib
from os import environ

from sqlalchemy import create_engine, func, desc, cast, JSON
from sqlalchemy import type_coerce
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.sql import union, select, and_, or_, not_, text
from sqlalchemy.sql import bindparam
from sqlalchemy.sql.functions import coalesce

# Setup

In [ ]:
driver = environ.get('SQL_DRIVER', '{ODBC Driver 17 for SQL Server}')
host = environ.get('SQL_HOST', 'sql-fabulous')
db = environ.get('SQL_DB', 'ScratchDB')
user = environ.get('SQL_USER', 'sa')
pw = environ.get('SQL_PASSWORD', 'HelloWorld1')

con_str = f'DRIVER={driver};SERVER={host};DATABASE={db};UID={user};PWD={pw}'

params = urllib.parse.quote_plus(con_str)  

# 'echo' emits generated sql
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", echo=True)

# Define Table Schema

In [ ]:
metadata = MetaData()

users = Table('users', metadata,
            Column('id', Integer, autoincrement=False, primary_key=True),
            Column('name', String(50), nullable=False),
            Column('fullname', String(255), nullable=False),
        )

addresses = Table('addresses', metadata,
                Column('id', Integer, autoincrement=False, primary_key=True),
                Column('user_id', None, ForeignKey('users.id')),
                Column('email_address', String(255), nullable=False)
            )

metadata.create_all(engine, checkfirst=True)

In [ ]:
# Establish connection
conn = engine.connect()

# Inserts

In [ ]:
ins = users.insert().values(id=1, name='jack', fullname='Jack Jones')
conn.execute(ins)

In [ ]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

In [ ]:
# Multi-insert
# Can also update(), delete()
conn.execute(addresses.insert(), [
        {'id': 1, 'user_id': 1, 'email_address' : 'jack@yahoo.com'},
        {'id': 2, 'user_id': 1, 'email_address' : 'jack@msn.com'},
        {'id': 3, 'user_id': 2, 'email_address' : 'www@www.org'},
        {'id': 4, 'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

# Select

### Make sure to close result cursors as below.

In [ ]:
s = select([users])
result = conn.execute(s) # returns cursor

for r in result:
    print(r)
    print(f"name: {r['name']}; fullname: {r['fullname']}")
    print(f"name: {r[1]}; fullname: {r[2]}")
    print(f"name: {r[users.c.name]}, fullname:{r[users.c.fullname]}")
          
result.close()

In [ ]:
# Explicit columns
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)

for row in result:
    print(row)
    
result.close()

In [ ]:
# Cross join
result = conn.execute(select([users, addresses]))
for row in result:
    print(row)
    
result.close()

In [ ]:
# Inner join
result = conn.execute(
            select([users, addresses])\
            .where(users.c.id == addresses.c.user_id)
        )
for row in result:
    print(row)
    
result.close()

# Operators
- "== None" becomes "IS / IS NOT NULL"

# Conjunctions

In [ ]:
stmt = select([(users.c.fullname + ", " + addresses.c.email_address).label('title')])\
        .where(and_(
            users.c.name.like('j%'),
            users.c.id == addresses.c.user_id,
            or_(
                addresses.c.email_address == 'wendy@aol.com',
                addresses.c.email_address == 'jack@yahoo.com'
            ),
            not_(users.c.id > 5)
        ))

result = conn.execute(stmt)

for r in result:
    print(r)
    
result.close()

In [ ]:
# Equivalent using multiple where clauses (method chaining)
stmt = select([(users.c.fullname + ", " + addresses.c.email_address).label('title')])\
        .where(users.c.name.like('j%'))\
        .where(users.c.id == addresses.c.user_id)\
        .where(or_(
                addresses.c.email_address == 'wendy@aol.com',
                addresses.c.email_address == 'jack@yahoo.com'
            ))\
        .where(not_(users.c.id > 5))

result = conn.execute(stmt)

for r in result:
    print(r)
    
result.close()

# Explicit SQL with Parameters

In [ ]:
# Explicit T-SQL
s = text(
    "SELECT users.fullname + ', ' + addresses.email_address AS title "
    "FROM users, addresses "
    "WHERE users.id = addresses.user_id "
        "AND users.name BETWEEN :x AND :y "
        "AND (addresses.email_address LIKE :e1 "
                "OR addresses.email_address LIKE :e2)")
# Or: s2 = s.bindparams(x='m', y='z', e1='%@aol.com', e2='%@msn.com')
r = conn.execute(s, x='m', y='z', e1='%@aol.com', e2='%@msn.com')

for row in r:
    print(row)
r.close()

# Group By / Order By

In [ ]:
stmt = select([addresses.c.user_id,
                func.count(addresses.c.id).label('num_addresses')]).\
        group_by(addresses.c.user_id)\
        .order_by(addresses.c.user_id, desc("num_addresses"))

conn.execute(stmt).fetchall()

# Table Aliasing / Sub-Queries

In [ ]:
a1 = addresses.alias('a1')
a2 = addresses.alias('a2')
s = select([users]).\
    where(and_(
            users.c.id == a1.c.user_id,
            users.c.id == a2.c.user_id,
            a1.c.email_address == 'jack@msn.com',
            a2.c.email_address == 'jack@yahoo.com'
    ))

conn.execute(s).fetchall()

In [ ]:
addresses_subq = s.alias()
s = select([users.c.name])\
    .where(users.c.id == addresses_subq.c.id)
conn.execute(s).fetchall()

# Joins

In [ ]:
# Can also use outerjoin()
s = select([users.c.fullname])\
        .select_from(
            users.join(addresses)
    )
conn.execute(s).fetchall()

In [ ]:
s = select([users, addresses.c.email_address])\
        .select_from(
            users.join(addresses,
                addresses.c.email_address.like(users.c.name + '%'))
    )
conn.execute(s).fetchall()

# Functions

In [ ]:
print(func.now())
print(func.current_timestamp())

In [ ]:
conn.execute(
    select([
        func.max(addresses.c.email_address, type_=String).
            label('maxemail')
        ])
).scalar()

# Window Functions

In [ ]:
s = select([
    users.c.id,
    func.row_number().over(order_by=users.c.name)
])

conn.execute(s).fetchall()

In [ ]:
s = select([
        users.c.id,
        func.count(addresses.c.id).over(
            partition_by=users.c.name,
            order_by=users.c.id,
            rows=(None, None)
        )
    ])\
    .select_from(
        users.join(addresses))
    
conn.execute(s).fetchall()

# Casts

In [ ]:
s = select([cast(users.c.id, String)])
conn.execute(s).fetchall()

In [ ]:
# Coerces but does not cast.
d = type_coerce({'some_key': {'foo': 'bar'}}, JSON)['some_key']
d

# Set Ops

In [ ]:
# Also, union_all(), intersect(), except_()
u = union(
    addresses.select()
        .where(addresses.c.email_address == 'foo@bar.com'),
    addresses.select()
        .where(addresses.c.email_address.like('%@yahoo.com')),
).order_by(addresses.c.email_address)

conn.execute(u).fetchall()

# Scalar Selects

In [ ]:
stmt = select([func.count(addresses.c.id)])\
        .where(users.c.id == addresses.c.user_id)\
        .as_scalar()

# This creates a correlated sub-query
conn.execute(select([users.c.name, stmt])).fetchall()

# This creates a named correlated sub-query
stmt = select([func.count(addresses.c.id)])\
        .where(users.c.id == addresses.c.user_id)\
        .label("address_count")

# This creates a correlated sub-query
conn.execute(select([users.c.name, stmt])).fetchall()

# Correlated sub-queries

In [ ]:
stmt = select([addresses.c.user_id])\
        .where(addresses.c.user_id == users.c.id)\
        .where(addresses.c.email_address == 'jack@yahoo.com')

# SqlAlchemy interprets the var as a sub-query
enclosing_stmt = select([users.c.name]).where(users.c.id == stmt)

conn.execute(enclosing_stmt).fetchall()

In [ ]:
# Specifying a specific correlated table when more than 1 is involved.
# Also, correlate_except()
stmt = select([users.c.id])\
        .where(users.c.id == addresses.c.user_id)\
        .where(users.c.name == 'jack')\
        .correlate(addresses)

enclosing_stmt = select([users.c.name, addresses.c.email_address])\
        .select_from(users.join(addresses))\
        .where(users.c.id == stmt)

conn.execute(enclosing_stmt).fetchall()

In [ ]:
# Disable correlation - creates non-correlated sub-query
stmt = select([users.c.id])\
        .where(users.c.name == 'wendy')\
        .correlate(None)

enclosing_stmt = select([users.c.name])\
        .where(users.c.id == stmt)

conn.execute(enclosing_stmt).fetchall()

# Group By, Order By, Offset Fetch

In [ ]:
stmt = select([users.c.name]).order_by(users.c.name.desc())
conn.execute(stmt).fetchall()

In [ ]:
stmt = select([users.c.name, func.count(addresses.c.id)])\
        .select_from(users.join(addresses))\
        .group_by(users.c.name)\
        .having(func.length(users.c.name) > 4)

conn.execute(stmt).fetchall()

In [ ]:
stmt = select([users.c.name])\
        .select_from(users.join(addresses))\
        .distinct()

conn.execute(stmt).fetchall()

In [ ]:
stmt = select([users.c.name, addresses.c.email_address])\
            .select_from(users.join(addresses))\
            .order_by(users.c.name.asc())\
            .limit(1).offset(1)

conn.execute(stmt).fetchall()

# Inserts, Updates, Deletes

In [ ]:
stmt = users.update()\
        .values(fullname="Fullname: " + users.c.name)

conn.execute(stmt)

In [ ]:
stmt = users.insert()\
        .values([
    {'id':7, 'name':'name1', 'fullname':'name1'},
    {'id':8, 'name':'name2', 'fullname':'name2'},
    {'id':9, 'name':'name3', 'fullname':'name3'},
])

conn.execute(stmt)

In [ ]:
# Override a parameter
stmt = users.insert()\
        .values(name=bindparam('_name') + " .. name")

conn.execute(stmt, [
    {'id':4, '_name':'name1', 'fullname':'name1'},
    {'id':5, '_name':'name2', 'fullname':'name2'},
    {'id':6, '_name':'name3', 'fullname':'name3'},
])

In [ ]:
stmt = users.update()\
        .where(users.c.name == 'jack')\
        .values(name='ed')

conn.execute(stmt)

In [ ]:
stmt = users.update()\
        .where(users.c.name == bindparam('oldname'))\
        .values(name=bindparam('newname'))

conn.execute(stmt, [
    {'oldname':'jack', 'newname':'jack2'},
    {'oldname':'wendy', 'newname':'wendy2'},
    {'oldname':'jim', 'newname':'jim2'},
])

In [ ]:
# Correlated update
stmt = users.update().\
        values(fullname="Name test2: "+addresses.c.email_address)\
        .where(users.c.id == addresses.c.id)\
        .where(addresses.c.email_address.endswith('.com'))

conn.execute(stmt)

In [ ]:
# Correlated update with explicit dictionary syntax
stmt = users.update().\
        values({
            users.c.name: "Name: " + addresses.c.email_address,
            users.c.fullname: "Email: " + addresses.c.email_address
        })\
        .where(users.c.id == addresses.c.id)\
        .where(addresses.c.email_address.endswith('.com'))

conn.execute(stmt)

In [ ]:
conn.execute(users.delete().where(users.c.fullname.startswith('jack')))

In [ ]:
# Subquery delete
stmt = users.delete()\
        .where(users.c.id.notin_(
            select([addresses.c.user_id])
        ))\
        .where(users.c.fullname.startswith('ed'))

conn.execute(stmt)